En esta parte se utilizará otro conjunto muy conocido de datos para aprendizaje, que consiste también de textos en inglés. Se trata de un conjunto de noticias de prensa de la Agencia Reuters, *Reuters-21578 *, https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

Las noticias han sido categorizadas a mano, con etiquetas de varios tipos, incluyendo *temas, lugares, organizaciones, personas* y otros criterios. Están en formato SGML, las etiquetas aparecen embebidas en el texto. Existe documentación en el archivo README de la distribución.

Nos interesaremos en el tipo de etiqueta *temas (en inglés en los textos topics)*. El objetivo es nuevamente aprender los temas a partir de los textos de las noticias, con la diferencia de que este es un problema multi-etiqueta : cada noticia puede tener varios temas. En vez de resolver el problema multi-etiqueta inicial (con más de 100 tópicos distintos) les pedimos que lo transformen según las siguientes simplificaciones: considere solo los 3 temas más frecuentes, y transforme el problema multi-etiqueta en 3 problemas de clasificación binaria

No se realiza para esta parte una especificación detallada, sino que les pedimos a Uds. que armen los pasos de una solución y definan el detalle del notebook para esta parte. Mínimamente se espera que procesen la entrada SGML, y encuentren algún modo de enfocar el problema multi-etiqueta en la versión simplificada. Se debe proponer algún modo de tratar el texto, con eventuales mejoras respecto a la vectorización de la parte 1. Se deben aplicar clasificadores y medir su performance, mínimamente con precision, recall y medida-F. Finalmente, se espera una discusión detallada de todo lo realizado y eventuales propuestas de mejora.

Puntos a tener en cuenta del conjunto de datos:
* En Topics dentro de la etiqueta <REUTERS> puede haber:
    * YES: Tiene al menos una categoría(puede existir algún error y no tener ningúna categoria)
    * NO: No tiene categoría(puede existir algún error y tener alguna categoria, supuestamente en esta versión no sucede)
    * BYPASS: No está indexado, no se verificó la categoría
* Hay etiquetas que dividen el conjunto en train y test
* Etiquetas:
    * <TOPICS>, </TOPICS> [ONCE, SAMELINE]: Lista de categorias. Si hay alguna estan delimitadas por <D></D>.
    * <UNKNOWN>, </UNKNOWN> metadatos desconocidos
    * <TEXT>, </TEXT> cuerpo del texto

In [36]:
import glob
#Cargo los archivos como lista de strings en documentos
lista_nombres_archivo = glob.glob('reuters21578/*.sgm')
documentos = []
for nombre_archivo in lista_nombres_archivo:
    with open (nombre_archivo, "r") as archivo:
        documento = archivo.read()
        documentos.append(documento)

In [37]:
#print documentos[0]

In [38]:
from BeautifulSoup import BeautifulSoup

raw_reuter_news = []
for documento in documentos:
    #coloco el contenido de los tags 'reuters' en otra lista: raw_reuter_news
    #esa lista contendra las noticias separadas en formato raw
    soup = BeautifulSoup(documento)
    for raw_reuter in soup('reuters'):
        raw_reuter_news.append(raw_reuter)

In [39]:
#verifico cantidad
print len(raw_reuter_news)

21578


In [40]:
raw_news_x_topics = {}
for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    for topics_soup in raw_reuter_soup.topics:
        #para cada topic en la lista de topics 
        #agrego la noticia a un diccionario clave topic y valor lista de noticias con ese topic.
        for topic_soup in topics_soup:
            try:
                #ya existe la lista.
                raw_news_x_topics[topic_soup].append(raw_reuter_soup)
            except KeyError:
                #es el primer valor
                raw_news_x_topics[topic_soup] = [raw_reuter_soup]

In [41]:
#ordeno el diccionario por cantidad de noticias en cada topic.
sorted_topics = sorted(raw_news_x_topics, key=lambda topic: len(raw_news_x_topics[topic]), reverse=True)
#los tres topics mas usados
primeros_topics = sorted_topics[:3]
for topic in primeros_topics:
        print topic,len(raw_news_x_topics[topic])

earn 3987
acq 2448
money-fx 801


In [42]:
#ya no necesito los metadatos
#separo en train y test segun el criterio lewissplit
#solo body
X_train = []
X_test = []
X_unused = []

#aca dejo info mas completa por si es necesario
#incluye fecha y titulo
X_train_plus = []
X_test_plus = []
X_unused_plus = []

#para etiqueta EARN
Y1_train = []
Y1_test = []
Y1_unused = []

#para etiquete ACQ
Y2_train = []
Y2_test = []
Y2_unused = []


#para etiqueta MONEY-FX
Y3_train = []
Y3_test = []
Y3_unused = []

for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    topics_de_la_noticia = []
    for topics_soup in raw_reuter_soup.topics:
        for topic_soup in topics_soup:
            #para cada topic en la lista de topics 
            topics_de_la_noticia.append(topic_soup)
    #veo si tiene las primeras categorias
    es_earn = 1 if u'earn' in topics_de_la_noticia else 0
    es_acq = 1 if u'acq' in topics_de_la_noticia else 0
    es_money_fx = 1 if u'money-fx' in topics_de_la_noticia else 0
    #extraigo contenido
    body = raw_reuter_soup.find('body').text if raw_reuter_soup.find('body') is not None else None
    title = raw_reuter_soup.find('title').text if raw_reuter_soup.find('title') is not None else None
    date = raw_reuter_soup.find('dateline').text if raw_reuter_soup.find('dateline') is not None else None
    #a veces no hay body
    if body is not None: 
        #conjunto entrenamiento
        if raw_reuter_soup['topics'] == 'YES':
            if raw_reuter_soup['lewissplit'] == 'TRAIN':
                X_train.append(body)
                X_train_plus.append('\n'.join([title,body,date]))
                Y1_train.append(es_earn)
                Y2_train.append(es_acq)
                Y3_train.append(es_money_fx)
            #conjunto test
            elif raw_reuter_soup['lewissplit'] == 'TEST':
                X_test.append(body)
                X_test_plus.append('\n'.join([title,body,date]))
                Y1_test.append(es_earn)
                Y2_test.append(es_acq)
                Y3_test.append(es_money_fx)
            #otro
            else:
                X_unused.append(body)
                X_unused_plus.append('\n'.join([title,body,date]))
                Y1_unused.append(es_earn)
                Y2_unused.append(es_acq)
                Y3_unused.append(es_money_fx)


In [43]:
#verifico cantidad noticias
news_n_topics = X_train + X_test + X_unused
print "Total: ", len(news_n_topics)
#verifico cantidad earn
print "EARN: ",sum(Y1_train+Y1_test+Y1_unused)
#verifico cantidad acq
print "ACQ: ",sum(Y2_train+Y2_test+Y2_unused)
#verifico cantidad money-fx
print "MONEY-FX", sum(Y3_train+Y3_test+Y3_unused)

len_X_train = len(X_train)
len_X_test = len(X_test)
len_X_unused = len(X_unused)

print "Train: ",len_X_train
print "Test: ",len_X_test
print "Unused: ",len_X_unused

assert(len(news_n_topics)==len_X_train+len_X_test+len_X_unused)

assert(len_X_train==len(Y1_train))
assert(len_X_train==len(Y2_train))
assert(len_X_train==len(Y3_train))

assert(len_X_test==len(Y1_test))
assert(len_X_test==len(Y2_test))
assert(len_X_test==len(Y3_test))

Total:  12344
EARN:  3775
ACQ:  2210
MONEY-FX 682
Train:  8762
Test:  3009
Unused:  573


In [47]:
#Organizacion del conjunto de entrenamiento
ConjuntoEntrenamiento = [
    ("EARN",Y1_train, Y2_test),
    ("ACQ",Y2_train, Y2_test),
    ("MONEY-FX",Y3_train, Y3_test)
]

In [48]:
#Vectorizacion
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect2 = CountVectorizer(min_df=1)
tidf_vect = TfidfVectorizer()
hash_vect = HashingVectorizer(non_negative=True)
Vectorizadores = [
    ("CountVectorizer", count_vect),
    ("CountVectorizer2", count_vect2),
    ("TfidfVectorizer", tidf_vect),
    ("HashingVectorizer", hash_vect)
]

In [49]:
#Comparación de diferentes clasificadores
#Fuente: http://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html

# Creación de clasificadores
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

"""
lr = LogisticRegression()
gnb = GaussianNB()
svc = LinearSVC(C=1.0)
rfc = RandomForestClassifier(n_estimators=100)
"""
multiNB = MultinomialNB()
sgd = SGDClassifier()
perceptron = Perceptron()
passiveaggclass = PassiveAggressiveClassifier()
passiveaggclass2 = PassiveAggressiveClassifier(loss='hinge',C=1.0)
passiveaggclass3 = PassiveAggressiveClassifier(loss='squared_hinge',C=1.0)

Clasificadores = [
    ("SGD", sgd),
    ("MultinomialNB", multiNB),
    ("Perceptron", perceptron),
    ("SGDClassifier", sgd),
    ("Passive-Aggressive I", passiveaggclass),
    ("Passive-Aggressive II", passiveaggclass2),
    ("Passive-Aggressive III", passiveaggclass3)
]

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import confusion_matrix
resultados = []

for nomclf,clasificador in Clasificadores:
    for nomvect, vectorizador in Vectorizadores:
        print "Metrica F1 con los siguientes parametros"
        print "Clasificador: " + nomclf
        pipeline = Pipeline([
              ('vectorizador', vectorizador),
              ('clasificador', clasificador)
            ])
        
        print "Vectorizador: " + nomvect 
        for nomcat,y_train,y_test in ConjuntoEntrenamiento:
            #X_V_train = vectorizador.transform(X_train)
            #Vectorizo test
            #X_V_test = vectorizador.transform(X_test)
            X_V_train = X_train
            X_V_test = X_test
            pipeline.fit(X_V_train, y_train)
            prediccion = pipeline.predict(X_V_test)  
            print "Categoria: " + nomcat
            print metrics.f1_score(y_test, prediccion, average='weighted')

Metrica F1 con los siguientes parametros
Clasificador: SGD
Vectorizador: CountVectorizer
Categoria: EARN
0.00243013365735
Categoria: ACQ
0.906095551895
Categoria: MONEY-FX
0.703125
Metrica F1 con los siguientes parametros
Clasificador: SGD
Vectorizador: CountVectorizer2
Categoria: EARN
0.00243013365735
Categoria: ACQ
0.906095551895
Categoria: MONEY-FX
0.703125
Metrica F1 con los siguientes parametros
Clasificador: SGD
Vectorizador: TfidfVectorizer
Categoria: EARN
0.00836820083682
Categoria: ACQ
0.931062449311
Categoria: MONEY-FX
0.743295019157
Metrica F1 con los siguientes parametros
Clasificador: SGD
Vectorizador: HashingVectorizer
Categoria: EARN

In [ ]:

X_V_train = vectorizer.fit_transform(X_train)
X_V_train.shape

In [44]:
#Vectorizo test
X_V_test = vectorizer.transform(X_test)
X_V_test.shape

(3009, 1048576)

In [1]:
#print Y1_train
#print X_V_train

In [49]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

clf = MultinomialNB()
clf2 = SGDClassifier()
clf3 = MultinomialNB()
#entreno Y1
clf.fit(X_V_train, Y1_train)
#predigo test
pred = clf.predict(X_V_test)
print "EARN: ",metrics.f1_score(Y1_test, pred, average='weighted')

#entreno Y2
clf2.fit(X_V_train, Y2_train)
#predigo test
pred2 = clf2.predict(X_V_test)
print "ACQ: ",metrics.f1_score(Y2_test, pred2, average='weighted')

#entreno Y3
clf3.fit(X_V_train, Y3_train)
#predigo test
pred3 = clf3.predict(X_V_test)
print "MONEY-FX: ",metrics.f1_score(Y3_test, pred3, average='weighted')

EARN:  0.929851510497
ACQ:  0.866375545852
MONEY-FX:  0.0
